<a href="https://colab.research.google.com/github/newcodevelop/FlutterExampleApps/blob/master/transfer_of_knowledge_from_english_to_hindi_language_and_vice_versa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!Author :- Dibyanayan Bandyopadhyay
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
file = '/content/gdrive/My Drive/multilingual grounded/final.csv'
df = pd.read_csv(file)

In [0]:
df

,Unnamed: 0,gold_label,hypo,hypo_hindi,premise,premise_hindi
0,0,neutral,A person is training his horse for a competition.,एक व्यक्ति अपने घोड़े को एक प्रतियोगिता के लिए...,A person on a horse jumps over a broken down a...,घोड़े पर सवार व्यक्ति टूटे हुए हवाई जहाज के ऊप...
1,1,contradiction,"A person is at a diner, ordering an omelette.","एक व्यक्ति एक भोजनशाला में है, एक आमलेट का ऑर्...",A person on a horse jumps over a broken down a...,घोड़े पर सवार व्यक्ति टूटे हुए हवाई जहाज के ऊप...
2,2,entailment,"A person is outdoors, on a horse.","एक व्यक्ति बाहर है, एक घोड़े पर।",A person on a horse jumps over a broken down a...,घोड़े पर सवार व्यक्ति टूटे हुए हवाई जहाज के ऊप...
3,3,neutral,They are smiling at their parents,वे अपने माता-पिता को देखकर मुस्कुरा रहे हैं,Children smiling and waving at camera,बच्चे मुस्कुराते हुए और कैमरे में लहराते हुए
4,4,entailment,There are children present,वहां बच्चे मौजूद हैं,Children smiling and waving at camera,बच्चे मुस्कुराते हुए और कैमरे में लहराते हुए
...,...,...,...,...,...,...
39993,39993,contradiction,A father holds his daughter's hand as they walk.,एक पिता अपनी बेटी का हाथ थामकर चलता है।,A father and son hold hands while walking.,चलते-चलते एक पिता और पुत्र हाथ पकड़ लेते हैं।
39994,39994,neutral,There is a group of people near the brown buil...,भूरे भवन के पास लोगों का एक समूह है,The man is walking near a brown building.,आदमी एक भूरे रंग की इमारत के पास चल रहा है।
39995,39995,entailment,There is a man near a brown building,एक भूरे रंग की इमारत के पास एक आदमी है,The man is walking near a brown building.,आदमी एक भूरे रंग की इमारत के पास चल रहा है।
39996,39996,contradiction,The man is sitting by the building,आदमी इमारत के पास बैठा है,The man is walking near a brown building.,आदमी एक भूरे रंग की इमारत के पास चल रहा है।


UTILITY FUNCTIONS



In [0]:
labels = list(df['gold_label'])
labs = []
for i in labels:
    if i=='contradiction':
        labs.append(0)
    elif i=='neutral':
        labs.append(1)
    elif i== 'entailment':
        labs.append(2)
    else:
        labs.append(3)
        


In [0]:
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
tf.enable_eager_execution()

TensorFlow 1.x selected.


In [0]:
tf.executing_eagerly()

True

In [0]:
lab = tf.keras.utils.to_categorical(labs)

In [0]:
premises,premises_hindi,hypos_hindi,hypos  = list(df['premise']),list(df['premise_hindi']),list(df['hypo_hindi']),list(df['hypo'])

In [0]:
hypos[0:10]

['A person is training his horse for a competition.',
 'A person is at a diner, ordering an omelette.',
 'A person is outdoors, on a horse.',
 'They are smiling at their parents',
 'There are children present',
 'The kids are frowning',
 'The boy skates down the sidewalk.',
 'The boy does a skateboarding trick.',
 'The boy is wearing safety equipment.',
 'An older man drinks his juice as he waits for his daughter to get off work.']

sen1,sen2,sen3,sen4 are 4 lists containing the word sequences of the english premise, english hypo, hindi premise and hindi hypo respectively

In [0]:
sen1,sen2,sen3,sen4 = [],[],[],[]

for i in premises:
    sen1.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in hypos:
    sen2.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in premises_hindi:
    sen3.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in hypos_hindi:
    sen4.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    


In [0]:
max_len_prem = 0 #...................... max length of english premise sentence
for i in sen1:
    if len(i)>max_len_prem:
        max_len_prem = len(i)
print(max_len_prem)

max_len_hypo = 0 #....................... max length of english hypothesis sentence
for i in sen2:
    if len(i)>max_len_hypo:
        max_len_hypo = len(i)
print(max_len_hypo)

max_len_prem_ = 0 # max length of hindi premise sentences
for i in sen3:
    if len(i)>max_len_prem_:
        max_len_prem_ = len(i)
print(max_len_prem_)

max_len_hypo_ = 0 # max length of hindi hypothesis sentences
for i in sen4:
    if len(i)>max_len_hypo_:
        max_len_hypo_ = len(i)
print(max_len_hypo_)

78
39
95
47


In [0]:
premises = sen1 #english prem
hypos = sen2 #english hypo
premises_ = sen3 #hindi premises
hypos_ = sen4 # hindi hypos

for sentences in range(len(premises)):
    #print(sentences)
    length = len(premises[sentences])
    max_len = max_len_prem
    diff = max_len-length
    for i in range(diff):
        premises[sentences].append('nan') # appending 'nan' token to english premises to make length of every sentences same

for sentences in range(len(hypos)):
    #print(sentences)
    length = len(hypos[sentences])
    max_len = max_len_hypo
    diff = max_len-length
    for i in range(diff):
        hypos[sentences].append('nan')  # appending 'nan' token to english hypos to make length of every sentences same

for sentences in range(len(premises_)):
    #print(sentences)
    length = len(premises_[sentences])
    max_len = max_len_prem_
    diff = max_len-length
    for i in range(diff):
        premises_[sentences].append('nan')  # appending 'nan' token to hindi premises to make length of every sentences same

for sentences in range(len(hypos_)):
    #print(sentences)
    length = len(hypos_[sentences])
    max_len = max_len_hypo_
    diff = max_len-length
    for i in range(diff):
        hypos_[sentences].append('nan')  # appending 'nan' token to hindi hypos to make length of every sentences same

In [0]:
from keras.preprocessing.text import Tokenizer
# generating padded docs to input in subsequent embedding layer
def generate_padded_docs(sentences):
    docs = sentences
  
    tok  = tf.keras.preprocessing.text.Tokenizer()
    tok.fit_on_texts(docs)
    docs = tok.texts_to_sequences(docs)
    #print(docs)
    max_len_multi = 0
    for i in docs:
        if len(i)>max_len_multi:
            max_len_multi = len(i)
    #print(max_len_multi)
    padded_docs = tf.keras.preprocessing.sequence.pad_sequences(docs, maxlen=max_len_multi, padding='post')
    vocab_size = max([max(i) for i in padded_docs])
    return padded_docs,max_len_multi,vocab_size

Using TensorFlow backend.


In [0]:
#generating padded docs from english premises and hypothesis
padded_docs_prem,max_len_multi_prem,vocab_size_prem = generate_padded_docs(premises)
padded_docs_hypo,max_len_multi_hypo,vocab_size_hypo = generate_padded_docs(hypos)

padded_docs_prem = np.asarray(padded_docs_prem,dtype = np.int32)
padded_docs_hypo = np.asarray(padded_docs_hypo,dtype = np.int32)


In [0]:
# generating padded docs from hindi premises and hypothesis
padded_docs_prem_,max_len_multi_prem_,vocab_size_prem_ = generate_padded_docs(premises_)
padded_docs_hypo_,max_len_multi_hypo_,vocab_size_hypo_ = generate_padded_docs(hypos_)

padded_docs_prem_ = np.asarray(padded_docs_prem_,dtype = np.int32)
padded_docs_hypo_ = np.asarray(padded_docs_hypo_,dtype = np.int32)

In [0]:
lab[0:35,:].shape

(35, 4)

In [0]:
import gensim
from gensim.models import Word2Vec
# word2vec on english premises,hypos and hindi premises,hypos
word_vector_length = 30 # hardcoding word vector length to 30
word2vec = Word2Vec(premises+hypos,size = word_vector_length, min_count=1)
word2vec_ = Word2Vec(premises_+hypos_,size = word_vector_length, min_count=1)

In [0]:
def fun1(docs,max_len):
    inp1 = np.zeros(shape = (39998,max_len,word_vector_length), dtype = np.float32)# 500 is no of examples placeholder for sentence_1(premise)
    #print(np.shape(inp1))

    for i in range(len(docs)):
        p = docs[i]
        #print(p)
        k=0
        for words in p:
            #print(words)
            inp1[i,k,:] = word2vec[words]

            k+=1
 
    return inp1
def fun2(docs,max_len):
    inp1 = np.zeros(shape = (39998,max_len,word_vector_length), dtype = np.float32)# 500 is no of examples placeholder for sentence_1(premise)
    print(np.shape(inp1))

    for i in range(len(docs)):
        p = docs[i]
        #print(p)
        k=0
        for words in p:
            #print(words)
            inp1[i,k,:] = word2vec_[words]

            k+=1
 
    return inp1
# utility function for defining word2vec models for english, hindi premises hypothesis to be input on subsequent neural net

In [0]:
inp1 = fun1(premises,max_len_prem) # input format for english premises
inp2 = fun1(hypos,max_len_hypo) # input format for english hypos
  
inp3 = fun2(premises_,max_len_prem_) #input format for hindi premises
inp4 = fun2(hypos_,max_len_hypo_) # input format for hindi hypos

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


(39998, 95, 30)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(39998, 47, 30)


In [0]:
# this keras model will be used to train english and hindi model using embedding generated above (no w2v)  and without other language's contxt
class MyModel1(tf.keras.Model):
    def __init__(self,word_vector_length,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo):
        super(MyModel1, self).__init__()
        self.lstm_shared = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(40,return_sequences = False,kernel_initializer = tf.keras.initializers.lecun_normal(seed=None),unit_forget_bias = True, name = 'lstm_main_1'),name = 'bid_1',merge_mode = 'mul')
        self.d1 = tf.keras.layers.Dense(10, activation='relu')
        self.d2 = tf.keras.layers.Dense(4, activation='softmax')
    
        self.word_vector_length = word_vector_length
        self.max_len_multi_prem = max_len_multi_prem
        self.max_len_multi_hypo = max_len_multi_hypo
        self.vocab_size_prem = vocab_size_prem
        self.vocab_size_hypo = vocab_size_hypo
        self.embedding1 = tf.keras.layers.Embedding(self.vocab_size_prem+1, self.word_vector_length, input_length=self.max_len_multi_prem)
        self.embedding2 = tf.keras.layers.Embedding(self.vocab_size_hypo+1, self.word_vector_length ,input_length=self.max_len_multi_hypo)
    def call(self,inputs):
      
        h1 = self.embedding1(inputs[0])
        x= self.lstm_shared(h1)
        #h1 = tf.nn.embedding_lookup(self.word_embeddings, inputs[1])#inputs[1] is padded_docs
        h2 = self.embedding2(inputs[1])
        print(np.shape(h1))
        
        y = self.lstm_shared(h2)
        z = tf.math.abs(x-y)

        
        z = self.d1(z)
        return self.d2(z)


In [0]:
# training on first 35k data of english language
model1 = MyModel1(30,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo)
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model1.fit([padded_docs_prem[0:35000,:],padded_docs_hypo[0:35000,:]],[np.asarray(lab[0:35000,:],dtype=np.int32)],epochs = 5)

(?, 78, 30)
Train on 35000 samples
Epoch 1/5
(?, 78, 30)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(?, 78, 30)
35000/35000 [==============================] - 263s 8ms/sample - loss: 1.0117 - acc: 0.4917
Epoch 2/5
35000/35000 [==============================] - 258s 7ms/sample - loss: 0.8354 - acc: 0.6316
Epoch 3/5
35000/35000 [==============================] - 261s 7ms/sample - loss: 0.7581 - acc: 0.6804
Epoch 4/5
35000/35000 [==============================] - 261s 7ms/sample - loss: 0.7018 - acc: 0.7055
Epoch 5/5
35000/35000 [==============================] - 261s 7ms/sample - loss: 0.6495 - acc: 0.7323


In [0]:
model1.evaluate([padded_docs_prem[35000:,:],padded_docs_hypo[35000:,:]],[lab[35000:,:]])
# evaluating remaining english sentences on the trained model

(?, 78, 30)
4998/1 [====================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.9400447106638066, 0.6010404]

In [0]:
#preparing english conext vector by obtaining from previously trained model1

x = model1.lstm_shared(model1.embedding1(padded_docs_prem))
y = model1.lstm_shared(model1.embedding2(padded_docs_hypo))
z = tf.math.abs(x-y)

z_ = model1.d1(z)
fin = model1.d2(z_)
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(fin,axis = 1),np.argmax(lab,axis = 1)))
print('shape of hidden layer {}'.format(np.shape(z)))
#obtaining hidden context from a trained lstm(net1) which was trained for rte with english sentence pairs.
hidden1 = np.asarray(z,dtype = np.float32)

0.7561878093904695
shape of hidden layer (39998, 40)


In [0]:
# training on first 35k data of hindi language
model1 = MyModel1(30,max_len_multi_prem_,max_len_multi_hypo_,vocab_size_prem_,vocab_size_hypo_)
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model1.fit([padded_docs_prem_[0:35000,:],padded_docs_hypo_[0:35000,:]],[lab[0:35000,:]],epochs = 5)

(?, 95, 30)
Train on 35000 samples
Epoch 1/5
(?, 95, 30)
(?, 95, 30)
35000/35000 [==============================] - 330s 9ms/sample - loss: 1.0098 - acc: 0.4948
Epoch 2/5
35000/35000 [==============================] - 322s 9ms/sample - loss: 0.8445 - acc: 0.6259
Epoch 3/5
35000/35000 [==============================] - 322s 9ms/sample - loss: 0.7676 - acc: 0.6724
Epoch 4/5
35000/35000 [==============================] - 321s 9ms/sample - loss: 0.7067 - acc: 0.7068
Epoch 5/5
35000/35000 [==============================] - 321s 9ms/sample - loss: 0.6742 - acc: 0.7245


In [0]:
model1.evaluate([padded_docs_prem_[35000:,:],padded_docs_hypo_[35000:,:]],[lab[35000:,:]])
# evaluating remaining hindi sentences on the trained model

(?, 95, 30)
4998/1 [====================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.9661509369601723, 0.5742297]

In [0]:
#preparing hindi conext vector by obtaining from previously trained model1
x = model1.lstm_shared(model1.embedding1(padded_docs_prem_))
y = model1.lstm_shared(model1.embedding2(padded_docs_hypo_))
z = tf.math.abs(x-y)

z_ = model1.d1(z)
fin = model1.d2(z_)
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(fin,axis = 1),np.argmax(lab,axis = 1)))
print('shape of hidden layer {}'.format(np.shape(z)))
#obtaining hidden context from a trained lstm(net1) which was trained for rte with hindi sentence pairs.
hidden2 = np.asarray(z,dtype = np.float32)

0.744537226861343
shape of hidden layer (39998, 40)


In [0]:
# redefining model 1 with a provision for accepting a context and adding it in final layer (IMPORTANT!)
class MyModel3(tf.keras.Model):
    def __init__(self,word_vector_length,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo):
        super(MyModel3, self).__init__()
        self.lstm_shared = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(40,return_sequences = False,kernel_initializer = tf.keras.initializers.lecun_normal(seed=None),unit_forget_bias = True, name = 'lstm_main_1'),name = 'bid_1',merge_mode = 'mul')
        self.d1 = tf.keras.layers.Dense(10, activation='relu')
        self.d2 = tf.keras.layers.Dense(4, activation='softmax')
        self.word_vector_length = word_vector_length
        self.max_len_multi_prem = max_len_multi_prem
        self.max_len_multi_hypo = max_len_multi_hypo
        self.vocab_size_prem = vocab_size_prem
        self.vocab_size_hypo = vocab_size_hypo
        self.embedding1 = tf.keras.layers.Embedding(self.vocab_size_prem+1, self.word_vector_length, input_length=self.max_len_multi_prem)
        self.embedding2 = tf.keras.layers.Embedding(self.vocab_size_hypo+1, self.word_vector_length ,input_length=self.max_len_multi_hypo)
        self.lateral_connection = tf.keras.layers.Dense(40,activation = 'relu')
    def call(self,inputs):
      
        h1 = self.embedding1(inputs[0])
        x= self.lstm_shared(h1)
        #h1 = tf.nn.embedding_lookup(self.word_embeddings, inputs[1])#inputs[1] is padded_docs
        h2 = self.embedding2(inputs[1])
        hidden1 = inputs[2]
        context = self.lateral_connection(hidden1)
        
        y = self.lstm_shared(h2)
        z = tf.math.abs(x-y)+context

        
        
        z = self.d1(z)
        return self.d2(z)


In [0]:
# transfering knowledge via progressive nets by fitting with context representation from net1........(hindi to english)
model_context = MyModel3(30,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo)
model_context.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_context.fit([padded_docs_prem[0:35000,:],padded_docs_hypo[0:35000,:],hidden2[0:35000,:]],[lab[0:35000,:]],epochs = 5)

Train on 35000 samples
Epoch 1/5
35000/35000 [==============================] - 269s 8ms/sample - loss: 0.6269 - acc: 0.7569
Epoch 2/5
35000/35000 [==============================] - 259s 7ms/sample - loss: 0.5653 - acc: 0.7773
Epoch 3/5
35000/35000 [==============================] - 260s 7ms/sample - loss: 0.5504 - acc: 0.7840
Epoch 4/5
35000/35000 [==============================] - 261s 7ms/sample - loss: 0.5317 - acc: 0.7917
Epoch 5/5
35000/35000 [==============================] - 261s 7ms/sample - loss: 0.5051 - acc: 0.8041


In [0]:
#evaluating the above model
model_context.evaluate([padded_docs_prem[35000:,:],padded_docs_hypo[35000:,:],hidden2[35000:,:]],[lab[35000:,:]])

4998/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.118052893922347, 0.5966387]

In [0]:
# transfering knowledge via progressive nets by fitting with context representation from net1........(english to hindi)
model_context = MyModel3(30,max_len_multi_prem_,max_len_multi_hypo_,vocab_size_prem_,vocab_size_hypo_)
model_context.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_context.fit([padded_docs_prem_[0:35000,:],padded_docs_hypo_[0:35000,:],hidden1[0:35000,:]],[lab[0:35000,:]],epochs = 5)

Train on 35000 samples
Epoch 1/5
35000/35000 [==============================] - 322s 9ms/sample - loss: 0.6017 - acc: 0.7648
Epoch 2/5
35000/35000 [==============================] - 317s 9ms/sample - loss: 0.5521 - acc: 0.7799
Epoch 3/5
35000/35000 [==============================] - 322s 9ms/sample - loss: 0.5315 - acc: 0.7900
Epoch 4/5
35000/35000 [==============================] - 322s 9ms/sample - loss: 0.4995 - acc: 0.8033
Epoch 5/5
35000/35000 [==============================] - 324s 9ms/sample - loss: 0.4631 - acc: 0.8193


In [0]:
#evaluating the above model
model_context.evaluate([padded_docs_prem_[35000:,:],padded_docs_hypo_[35000:,:],hidden1[35000:,:]],[lab[35000:,:]])

4998/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.1349535552488894, 0.60664266]